In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import prince

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv("historical-correct.csv")
df.head()

,ID,Date,Venue,T1,T2,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,...,T2_BowlsFace,T2_OversBowl,T2_RunsScored,T2_4s,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner
0,1,9-Jan-11,Durban,South Africa,India,34,3835,655.3,4541,385,...,2574,428.2,3297,277,132,3421,118,1.000,136,T2
1,2,12-Jan-11,Adelaide,Australia,England,36,3956,674.1,5283,440,...,3483,546.4,4390,418,120,4207,177,1.142,188,T2
2,3,14-Jan-11,Melbourne,Australia,England,37,4076,694.1,5436,454,...,3603,566.4,4541,435,124,4362,181,1.214,197,T1
3,4,21-Apr-11,Gros Islet,Pakistan,West Indies,38,4464,717.1,5313,445,...,2525,445.4,3002,276,99,3603,128,0.714,153,T2
4,5,4-Jun-11,Port of Spain,India,West Indies,23,2694,448.2,3461,293,...,2646,465.4,3146,289,105,3745,136,0.785,160,T1


## data pre-processing

In [3]:
df.shape

(519, 26)

In [4]:
df.dtypes

ID                   int64
Date                object
Venue               object
T1                  object
T2                  object
T1_Mat               int64
T1_BowlsFace         int64
T1_OversBowl       float64
T1_RunsScored        int64
T1_4s                int64
T1_6s                int64
T1_RunsConceded      int64
T1_WktsTaken         int64
T1_W/L             float64
T1_WktsLost          int64
T2_Mat               int64
T2_BowlsFace         int64
T2_OversBowl       float64
T2_RunsScored        int64
T2_4s                int64
T2_6s                int64
T2_RunsConceded      int64
T2_WktsTaken         int64
T2_W/L             float64
T2_WktsLost          int64
Winner              object
dtype: object

In [220]:
#  missing values
df.isnull().sum()

ID                 0
Date               0
Venue              0
T1                 0
T2                 0
T1_Mat             0
T1_BowlsFace       0
T1_OversBowl       0
T1_RunsScored      0
T1_4s              0
T1_6s              0
T1_RunsConceded    0
T1_WktsTaken       0
T1_W/L             0
T1_WktsLost        0
T2_Mat             0
T2_BowlsFace       0
T2_OversBowl       0
T2_RunsScored      0
T2_4s              0
T2_6s              0
T2_RunsConceded    0
T2_WktsTaken       0
T2_W/L             0
T2_WktsLost        0
Winner             0
dtype: int64

In [221]:
# duplicates
df.duplicated().sum()

0

In [5]:
# remove unwanted columns
df1 = df.drop(['Date', 'T1', 'T2'], axis=1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_BowlsFace,T2_OversBowl,T2_RunsScored,T2_4s,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner
0,1,Durban,34,3835,655.3,4541,385,157,4763,217,...,2574,428.2,3297,277,132,3421,118,1.000,136,T2
1,2,Adelaide,36,3956,674.1,5283,440,208,4978,235,...,3483,546.4,4390,418,120,4207,177,1.142,188,T2
2,3,Melbourne,37,4076,694.1,5436,454,212,5131,244,...,3603,566.4,4541,435,124,4362,181,1.214,197,T1
3,4,Gros Islet,38,4464,717.1,5313,445,177,5151,235,...,2525,445.4,3002,276,99,3603,128,0.714,153,T2
4,5,Port of Spain,23,2694,448.2,3461,293,137,3561,126,...,2646,465.4,3146,289,105,3745,136,0.785,160,T1


In [6]:
# Convert T1/T2 string columns into int columns of 1/0
df1[['Winner']] = \
(df1[['Winner']] == 'T1').astype(int)
df1.tail()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_BowlsFace,T2_OversBowl,T2_RunsScored,T2_4s,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner
514,515,Christchurch,166,19340,3163.1,23555,2084,738,23685,978,...,16013,2655.2,20362,1665,833,20895,840,0.958,915,0
515,516,Christchurch,142,16110,2675.2,20482,1671,839,21025,847,...,11179,1788.1,12475,1134,338,14237,492,0.293,714,1
516,517,Canberra,152,17391,2885.0,23285,2140,873,23412,881,...,17235,2980.3,22802,1921,887,22966,971,1.150,936,1
517,518,Christchurch,167,19460,3179.2,23675,2099,738,23811,979,...,11299,1808.1,12626,1151,340,14440,497,0.289,721,1
518,519,Christchurch,168,19580,3199.2,23846,2117,739,23984,984,...,16230,2695.2,20680,1684,850,21179,854,0.986,921,1


In [7]:
df1['Winner'].value_counts()

1    298
0    221
Name: Winner, dtype: int64

## feature engineering

In [8]:
# create overs batted var
df1['T1_OversBat'] = 0
for i in range(0, len(df1)):
    if str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.167' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6)
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.333' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6)
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.5' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6) 
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.667' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6) 
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.833' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6) 
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.0' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6)
        
df1.head()

df1['T2_OversBat'] = 0
for i in range(0, len(df1)):
    if str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.167' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.333' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.5' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.667' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.833' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.0' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6)
        
df1.head(10)

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_RunsScored,T2_4s,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner,T1_OversBat,T2_OversBat
0,1,Durban,34,3835,655.3,4541,385,157,4763,217,...,3297,277,132,3421,118,1.000,136,0,639.166667,429.000000
1,2,Adelaide,36,3956,674.1,5283,440,208,4978,235,...,4390,418,120,4207,177,1.142,188,0,659.333333,580.500000
2,3,Melbourne,37,4076,694.1,5436,454,212,5131,244,...,4541,435,124,4362,181,1.214,197,1,679.333333,600.500000
3,4,Gros Islet,38,4464,717.1,5313,445,177,5151,235,...,3002,276,99,3603,128,0.714,153,0,744.000000,420.833333
4,5,Port of Spain,23,2694,448.2,3461,293,137,3561,126,...,3146,289,105,3745,136,0.785,160,1,449.000000,441.000000
5,6,Bristol,32,3725,586.4,4674,442,125,4507,188,...,3882,380,93,4100,167,1.071,183,0,620.833333,550.666667
6,7,Pallekele,38,4196,714.1,5580,463,217,5272,250,...,4005,394,94,4233,174,1.142,184,0,699.333333,568.000000
7,8,Pallekele,39,4316,734.1,5737,475,223,5463,253,...,4190,418,99,4395,181,1.214,187,0,719.333333,588.000000
8,9,Manchester,24,2816,468.2,3607,303,143,3700,131,...,4803,450,130,4641,189,1.062,212,0,469.333333,641.000000
9,10,The Oval,34,3963,623.4,4959,469,132,4799,198,...,3282,301,110,3903,142,0.733,165,1,660.500000,461.000000


In [9]:
# create overs bowl var

df1['T1_OversBowled'] = 0

for i in range(0, len(df1)):
    if str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.1' :
        df1['T1_OversBowled'][i]=  int(df1['T1_OversBowl'][i])+ 0.166667
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.2' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.333333
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.3' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.500000
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.4' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.666667
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.5' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.833333
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.0' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i]) 
 
 
df1['T2_OversBowled'] = 0
    
for i in range(0, len(df1)):
    if str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.1' :
        df1['T2_OversBowled'][i]=  int(df1['T2_OversBowl'][i])+ 0.166667
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.2' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.333333
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.3' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.500000
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.4' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.666667
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.5' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.833333
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.0' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i]) 
        
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner,T1_OversBat,T2_OversBat,T1_OversBowled,T2_OversBowled
0,1,Durban,34,3835,655.3,4541,385,157,4763,217,...,132,3421,118,1.000,136,0,639.166667,429.000000,655.500000,428.333333
1,2,Adelaide,36,3956,674.1,5283,440,208,4978,235,...,120,4207,177,1.142,188,0,659.333333,580.500000,674.166667,546.666667
2,3,Melbourne,37,4076,694.1,5436,454,212,5131,244,...,124,4362,181,1.214,197,1,679.333333,600.500000,694.166667,566.666667
3,4,Gros Islet,38,4464,717.1,5313,445,177,5151,235,...,99,3603,128,0.714,153,0,744.000000,420.833333,717.166667,445.666667
4,5,Port of Spain,23,2694,448.2,3461,293,137,3561,126,...,105,3745,136,0.785,160,1,449.000000,441.000000,448.333333,465.666667


In [10]:
# Venue---> Continent
df['Venue'].unique()

array(['Durban', 'Adelaide', 'Melbourne', 'Gros Islet', 'Port of Spain',
       'Bristol', 'Pallekele', 'Manchester', 'The Oval', 'Mirpur',
       'Cape Town', 'Johannesburg', 'Eden Gardens', 'Abu Dhabi', 'Sydney',
       'Wellington', 'Hamilton', 'Auckland', 'Dubai (DSC)', 'Bridgetown',
       'Hambantota', 'Nottingham', 'Lauderhill', 'Chester-le-Street',
       'Chennai', 'Birmingham', 'Colombo (RPS)', 'Pune', 'Wankhede',
       'East London', 'Bengaluru', 'Gqeberha', 'Ahmedabad', 'Brisbane',
       'Centurion', 'Kingstown', 'Southampton', 'Rajkot', 'Hobart',
       'Chattogram', 'Roseau', 'Cardiff', 'Dharamsala', 'Cuttack',
       'Mount Maunganui', 'Ranchi', 'Visakhapatnam', 'Nagpur', 'Mohali',
       'Delhi', 'Napier', 'Kanpur', 'Geelong', 'Taunton', 'Kingston',
       'Guwahati', 'Bloemfontein', 'Potchefstroom', 'Lahore',
       'Thiruvananthapuram', 'Indore', 'Nelson', 'Sylhet', 'Karachi',
       'Harare', 'Basseterre', 'Lucknow', 'Carrara', 'Providence',
       'Christchurch', 

In [11]:
# 1. Venue var
Oceania = ['Sydney', 'Melbourne', 'Auckland', 'Hamilton', 'Brisbane','Wellington', 'Hobart', 'Adelaide', 'Mount Maunganui',
           'Napier', 'Geelong', 'Nelson', 'Carrara', 'Christchurch', 'Canberra', 'Perth', 'Dunedin']

Africa = ['Centurion', 'Johannesburg', 'Cape Town', 'Durban', 'Bloemfontein', 'Potchefstroom', 'Harare', 'East London',
         'Gqeberha', 'Paarl', "St George's"]

Asia = ['Pallekele', 'Colombo (RPS)', 'Hambantota', 'Rajkot', 'Mirpur', 'Dubai (DSC)', 'Chattogram', 'Dharamsala',
        'Cuttack', 'Pune', 'Ranchi', 'Visakhapatnam','Nagpur', 'Eden Gardens', 'Wankhede', 'Bengaluru', 'Mohali',
       'Delhi', 'Abu Dhabi', 'Kanpur', 'Guwahati', 'Lahore', 'Thiruvananthapuram', 'Indore', 'Sylhet', 'Karachi',
       'Lucknow', 'Chennai', 'Hyderabad (Deccan)', 'Ahmedabad', 'Sharjah', 'Jaipur']

Europe = ['The Oval', 'Southampton', 'Chester-le-Street', 'Birmingham', 'Manchester', 'Cardiff', 'Taunton', 'Bristol', 
          'Nottingham', 'Leeds']

America = ['Kingstown', 'Bridgetown', 'Roseau', 'Lauderhill', 'Port of Spain', 'Kingston', 'Basseterre', 'Gros Islet', 
           'Providence', 'Coolidge', 'Tarouba']

for i in range(0, len(df1)):
    if df1['Venue'][i] in Oceania:
        df1['Venue'][i]='Oceania'
    elif df1['Venue'][i]  in Africa:
        df1['Venue'][i] = 'Africa'
    elif df1['Venue'][i]  in Asia:
        df1['Venue'][i] = 'Asia'
    elif df1['Venue'][i]  in Europe:
        df1['Venue'][i] = 'Europe'
    elif df1['Venue'][i]  in America:
        df1['Venue'][i] = 'America'
        
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner,T1_OversBat,T2_OversBat,T1_OversBowled,T2_OversBowled
0,1,Africa,34,3835,655.3,4541,385,157,4763,217,...,132,3421,118,1.000,136,0,639.166667,429.000000,655.500000,428.333333
1,2,Oceania,36,3956,674.1,5283,440,208,4978,235,...,120,4207,177,1.142,188,0,659.333333,580.500000,674.166667,546.666667
2,3,Oceania,37,4076,694.1,5436,454,212,5131,244,...,124,4362,181,1.214,197,1,679.333333,600.500000,694.166667,566.666667
3,4,America,38,4464,717.1,5313,445,177,5151,235,...,99,3603,128,0.714,153,0,744.000000,420.833333,717.166667,445.666667
4,5,America,23,2694,448.2,3461,293,137,3561,126,...,105,3745,136,0.785,160,1,449.000000,441.000000,448.333333,465.666667


In [12]:
# 2. avg runs scored per over var
df1['T1_AvgRunsScored/Over'] = df1.apply(lambda row: np.round(row.T1_RunsScored/row.T1_OversBat,2) , axis = 1)
df1['T2_AvgRunsScored/Over'] = df1.apply(lambda row: np.round(row.T2_RunsScored/row.T2_OversBat,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner,T1_OversBat,T2_OversBat,T1_OversBowled,T2_OversBowled,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over
0,1,Africa,34,3835,655.3,4541,385,157,4763,217,...,118,1.000,136,0,639.166667,429.000000,655.500000,428.333333,7.10,7.69
1,2,Oceania,36,3956,674.1,5283,440,208,4978,235,...,177,1.142,188,0,659.333333,580.500000,674.166667,546.666667,8.01,7.56
2,3,Oceania,37,4076,694.1,5436,454,212,5131,244,...,181,1.214,197,1,679.333333,600.500000,694.166667,566.666667,8.00,7.56
3,4,America,38,4464,717.1,5313,445,177,5151,235,...,128,0.714,153,0,744.000000,420.833333,717.166667,445.666667,7.14,7.13
4,5,America,23,2694,448.2,3461,293,137,3561,126,...,136,0.785,160,1,449.000000,441.000000,448.333333,465.666667,7.71,7.13


In [13]:
# 3. avg runs conceded per over var
df1['T1_AvgRunsConceded/Over'] = df1.apply(lambda row: np.round(row.T1_RunsConceded/row.T1_OversBowled,2) , axis = 1)
df1['T2_AvgRunsConceded/Over'] = df1.apply(lambda row: np.round(row.T2_RunsConceded/row.T2_OversBowled,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_WktsLost,Winner,T1_OversBat,T2_OversBat,T1_OversBowled,T2_OversBowled,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over
0,1,Africa,34,3835,655.3,4541,385,157,4763,217,...,136,0,639.166667,429.000000,655.500000,428.333333,7.10,7.69,7.27,7.99
1,2,Oceania,36,3956,674.1,5283,440,208,4978,235,...,188,0,659.333333,580.500000,674.166667,546.666667,8.01,7.56,7.38,7.70
2,3,Oceania,37,4076,694.1,5436,454,212,5131,244,...,197,1,679.333333,600.500000,694.166667,566.666667,8.00,7.56,7.39,7.70
3,4,America,38,4464,717.1,5313,445,177,5151,235,...,153,0,744.000000,420.833333,717.166667,445.666667,7.14,7.13,7.18,8.08
4,5,America,23,2694,448.2,3461,293,137,3561,126,...,160,1,449.000000,441.000000,448.333333,465.666667,7.71,7.13,7.94,8.04


In [14]:
# 4. create avg boundaries earned per over var
df1['T1_Bound'] = df1.apply(lambda row: np.round((row.T1_4s+row.T1_6s)/row.T1_OversBat,2) , axis = 1)
df1['T2_Bound'] = df1.apply(lambda row: np.round((row.T2_4s+row.T2_6s)/row.T2_OversBat,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T1_OversBat,T2_OversBat,T1_OversBowled,T2_OversBowled,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over,T1_Bound,T2_Bound
0,1,Africa,34,3835,655.3,4541,385,157,4763,217,...,639.166667,429.000000,655.500000,428.333333,7.10,7.69,7.27,7.99,0.85,0.95
1,2,Oceania,36,3956,674.1,5283,440,208,4978,235,...,659.333333,580.500000,674.166667,546.666667,8.01,7.56,7.38,7.70,0.98,0.93
2,3,Oceania,37,4076,694.1,5436,454,212,5131,244,...,679.333333,600.500000,694.166667,566.666667,8.00,7.56,7.39,7.70,0.98,0.93
3,4,America,38,4464,717.1,5313,445,177,5151,235,...,744.000000,420.833333,717.166667,445.666667,7.14,7.13,7.18,8.08,0.84,0.89
4,5,America,23,2694,448.2,3461,293,137,3561,126,...,449.000000,441.000000,448.333333,465.666667,7.71,7.13,7.94,8.04,0.96,0.89


In [15]:
# 5. avg wickets taken per over
df1['T1_Avg_Wkts_Taken'] = df1.apply(lambda row: np.round(row.T1_WktsTaken/row.T1_OversBowled,2) , axis = 1)
df1['T2_Avg_Wkts_Taken'] = df1.apply(lambda row: np.round(row.T2_WktsTaken/row.T2_OversBowled,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T1_OversBowled,T2_OversBowled,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over,T1_Bound,T2_Bound,T1_Avg_Wkts_Taken,T2_Avg_Wkts_Taken
0,1,Africa,34,3835,655.3,4541,385,157,4763,217,...,655.500000,428.333333,7.10,7.69,7.27,7.99,0.85,0.95,0.33,0.28
1,2,Oceania,36,3956,674.1,5283,440,208,4978,235,...,674.166667,546.666667,8.01,7.56,7.38,7.70,0.98,0.93,0.35,0.32
2,3,Oceania,37,4076,694.1,5436,454,212,5131,244,...,694.166667,566.666667,8.00,7.56,7.39,7.70,0.98,0.93,0.35,0.32
3,4,America,38,4464,717.1,5313,445,177,5151,235,...,717.166667,445.666667,7.14,7.13,7.18,8.08,0.84,0.89,0.33,0.29
4,5,America,23,2694,448.2,3461,293,137,3561,126,...,448.333333,465.666667,7.71,7.13,7.94,8.04,0.96,0.89,0.28,0.29


In [16]:
# 6. avg wickets lost
df1['T1_Avg_Wkts_Lost'] = df1.apply(lambda row: np.round(row.T1_WktsLost/row.T1_OversBat,2) , axis = 1)
df1['T2_Avg_Wkts_Lost'] = df1.apply(lambda row: np.round(row.T2_WktsLost/row.T2_OversBat,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over,T1_Bound,T2_Bound,T1_Avg_Wkts_Taken,T2_Avg_Wkts_Taken,T1_Avg_Wkts_Lost,T2_Avg_Wkts_Lost
0,1,Africa,34,3835,655.3,4541,385,157,4763,217,...,7.10,7.69,7.27,7.99,0.85,0.95,0.33,0.28,0.32,0.32
1,2,Oceania,36,3956,674.1,5283,440,208,4978,235,...,8.01,7.56,7.38,7.70,0.98,0.93,0.35,0.32,0.33,0.32
2,3,Oceania,37,4076,694.1,5436,454,212,5131,244,...,8.00,7.56,7.39,7.70,0.98,0.93,0.35,0.32,0.33,0.33
3,4,America,38,4464,717.1,5313,445,177,5151,235,...,7.14,7.13,7.18,8.08,0.84,0.89,0.33,0.29,0.34,0.36
4,5,America,23,2694,448.2,3461,293,137,3561,126,...,7.71,7.13,7.94,8.04,0.96,0.89,0.28,0.29,0.32,0.36


In [17]:
df1.columns

Index(['ID', 'Venue', 'T1_Mat', 'T1_BowlsFace', 'T1_OversBowl',
       'T1_RunsScored', 'T1_4s', 'T1_6s', 'T1_RunsConceded', 'T1_WktsTaken',
       'T1_W/L', 'T1_WktsLost', 'T2_Mat', 'T2_BowlsFace', 'T2_OversBowl',
       'T2_RunsScored', 'T2_4s', 'T2_6s', 'T2_RunsConceded', 'T2_WktsTaken',
       'T2_W/L', 'T2_WktsLost', 'Winner', 'T1_OversBat', 'T2_OversBat',
       'T1_OversBowled', 'T2_OversBowled', 'T1_AvgRunsScored/Over',
       'T2_AvgRunsScored/Over', 'T1_AvgRunsConceded/Over',
       'T2_AvgRunsConceded/Over', 'T1_Bound', 'T2_Bound', 'T1_Avg_Wkts_Taken',
       'T2_Avg_Wkts_Taken', 'T1_Avg_Wkts_Lost', 'T2_Avg_Wkts_Lost'],
      dtype='object')

In [18]:
# remove unvanted columns
data = df1.drop([ 
       'T1_BowlsFace', 'T1_OversBowl',
       'T1_RunsScored', 'T1_4s', 'T1_6s', 'T1_RunsConceded', 'T1_WktsTaken',
       'T1_WktsLost',   'T2_BowlsFace', 'T2_OversBowl',
       'T2_RunsScored', 'T2_4s', 'T2_6s', 'T2_RunsConceded', 'T2_WktsTaken',
       'T2_WktsLost',   'T1_OversBat', 'T2_OversBat',
       'T1_OversBowled', 'T2_OversBowled',  
], axis=1)
data.head()

,ID,Venue,T1_Mat,T1_W/L,T2_Mat,T2_W/L,Winner,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over,T1_Bound,T2_Bound,T1_Avg_Wkts_Taken,T2_Avg_Wkts_Taken,T1_Avg_Wkts_Lost,T2_Avg_Wkts_Lost
0,1,Africa,34,1.833,22,1.000,0,7.10,7.69,7.27,7.99,0.85,0.95,0.33,0.28,0.32,0.32
1,2,Oceania,36,1.400,30,1.142,0,8.01,7.56,7.38,7.70,0.98,0.93,0.35,0.32,0.33,0.32
2,3,Oceania,37,1.312,31,1.214,1,8.00,7.56,7.39,7.70,0.98,0.93,0.35,0.32,0.33,0.33
3,4,America,38,1.235,24,0.714,0,7.14,7.13,7.18,8.08,0.84,0.89,0.33,0.29,0.34,0.36
4,5,America,23,1.090,25,0.785,1,7.71,7.13,7.94,8.04,0.96,0.89,0.28,0.29,0.32,0.36


In [19]:
data.shape

(519, 17)

In [20]:
data.columns

Index(['ID', 'Venue', 'T1_Mat', 'T1_W/L', 'T2_Mat', 'T2_W/L', 'Winner',
       'T1_AvgRunsScored/Over', 'T2_AvgRunsScored/Over',
       'T1_AvgRunsConceded/Over', 'T2_AvgRunsConceded/Over', 'T1_Bound',
       'T2_Bound', 'T1_Avg_Wkts_Taken', 'T2_Avg_Wkts_Taken',
       'T1_Avg_Wkts_Lost', 'T2_Avg_Wkts_Lost'],
      dtype='object')

In [21]:
# arrange the columns
data = data.loc[:,['ID',  'Venue', 
             'T1_Mat', 'T1_AvgRunsScored/Over', 'T1_AvgRunsConceded/Over', 'T1_Avg_Wkts_Taken', 'T1_Avg_Wkts_Lost', 'T1_Bound', 'T1_W/L',
             'T2_Mat', 'T2_AvgRunsScored/Over', 'T2_AvgRunsConceded/Over', 'T2_Avg_Wkts_Taken', 'T2_Avg_Wkts_Lost', 'T2_Bound','T2_W/L', 
            'Winner'
           ]]
data.head()

,ID,Venue,T1_Mat,T1_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T1_Avg_Wkts_Taken,T1_Avg_Wkts_Lost,T1_Bound,T1_W/L,T2_Mat,T2_AvgRunsScored/Over,T2_AvgRunsConceded/Over,T2_Avg_Wkts_Taken,T2_Avg_Wkts_Lost,T2_Bound,T2_W/L,Winner
0,1,Africa,34,7.10,7.27,0.33,0.32,0.85,1.833,22,7.69,7.99,0.28,0.32,0.95,1.000,0
1,2,Oceania,36,8.01,7.38,0.35,0.33,0.98,1.400,30,7.56,7.70,0.32,0.32,0.93,1.142,0
2,3,Oceania,37,8.00,7.39,0.35,0.33,0.98,1.312,31,7.56,7.70,0.32,0.33,0.93,1.214,1
3,4,America,38,7.14,7.18,0.33,0.34,0.84,1.235,24,7.13,8.08,0.29,0.36,0.89,0.714,0
4,5,America,23,7.71,7.94,0.28,0.32,0.96,1.090,25,7.13,8.04,0.29,0.36,0.89,0.785,1


In [239]:
data.shape

(519, 17)

In [240]:
# save the dataset
data.to_csv('historical-featured.csv')

## APPENDIX

In [ ]:
df = pd.read_csv("historical-correct.csv")
# remove unwanted columns
df1 = df.drop(['Date', 'T1', 'T2'], axis=1)
# Convert T1/T2 string columns into int columns of 1/0
df1[['Winner']] = \
(df1[['Winner']] == 'T1').astype(int)
# create overs batted var
df1['T1_OversBat'] = 0
for i in range(0, len(df1)):
    if str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.167' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6)
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.333' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6)
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.5' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6) 
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.667' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6) 
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.833' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6) 
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.0' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6)
df1['T2_OversBat'] = 0
for i in range(0, len(df1)):
    if str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.167' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.333' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.5' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.667' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.833' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.0' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6)
# create overs bowl var
df1['T1_OversBowled'] = 0
for i in range(0, len(df1)):
    if str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.1' :
        df1['T1_OversBowled'][i]=  int(df1['T1_OversBowl'][i])+ 0.166667
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.2' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.333333
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.3' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.500000
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.4' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.666667
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.5' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.833333
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.0' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])
for i in range(0, len(df1)):
    if str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.1' :
        df1['T2_OversBowled'][i]=  int(df1['T2_OversBowl'][i])+ 0.166667
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.2' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.333333
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.3' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.500000
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.4' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.666667
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.5' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.833333
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.0' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i]) 
# 1. Venue var
Oceania = ['Sydney', 'Melbourne', 'Auckland', 'Hamilton', 'Brisbane','Wellington', 'Hobart', 'Adelaide', 'Mount Maunganui',
           'Napier', 'Geelong', 'Nelson', 'Carrara', 'Christchurch', 'Canberra', 'Perth', 'Dunedin']
Africa = ['Centurion', 'Johannesburg', 'Cape Town', 'Durban', 'Bloemfontein', 'Potchefstroom', 'Harare', 'East London',
         'Gqeberha', 'Paarl', "St George's"]
Asia = ['Pallekele', 'Colombo (RPS)', 'Hambantota', 'Rajkot', 'Mirpur', 'Dubai (DSC)', 'Chattogram', 'Dharamsala',
        'Cuttack', 'Pune', 'Ranchi', 'Visakhapatnam','Nagpur', 'Eden Gardens', 'Wankhede', 'Bengaluru', 'Mohali',
       'Delhi', 'Abu Dhabi', 'Kanpur', 'Guwahati', 'Lahore', 'Thiruvananthapuram', 'Indore', 'Sylhet', 'Karachi',
       'Lucknow', 'Chennai', 'Hyderabad (Deccan)', 'Ahmedabad', 'Sharjah', 'Jaipur']
Europe = ['The Oval', 'Southampton', 'Chester-le-Street', 'Birmingham', 'Manchester', 'Cardiff', 'Taunton', 'Bristol', 
          'Nottingham', 'Leeds']
America = ['Kingstown', 'Bridgetown', 'Roseau', 'Lauderhill', 'Port of Spain', 'Kingston', 'Basseterre', 'Gros Islet', 
           'Providence', 'Coolidge', 'Tarouba']
for i in range(0, len(df1)):
    if df1['Venue'][i] in Oceania:
        df1['Venue'][i]='Oceania'
    elif df1['Venue'][i]  in Africa:
        df1['Venue'][i] = 'Africa'
    elif df1['Venue'][i]  in Asia:
        df1['Venue'][i] = 'Asia'
    elif df1['Venue'][i]  in Europe:
        df1['Venue'][i] = 'Europe'
    elif df1['Venue'][i]  in America:
        df1['Venue'][i] = 'America'
# 2. avg runs scored per over var
df1['T1_AvgRunsScored/Over'] = df1.apply(lambda row: np.round(row.T1_RunsScored/row.T1_OversBat,2) , axis = 1)
df1['T2_AvgRunsScored/Over'] = df1.apply(lambda row: np.round(row.T2_RunsScored/row.T2_OversBat,2) , axis = 1)
# 3. avg runs conceded per over var
df1['T1_AvgRunsConceded/Over'] = df1.apply(lambda row: np.round(row.T1_RunsConceded/row.T1_OversBowled,2) , axis = 1)
df1['T2_AvgRunsConceded/Over'] = df1.apply(lambda row: np.round(row.T2_RunsConceded/row.T2_OversBowled,2) , axis = 1)
# 4. create avg boundaries earned per over var
df1['T1_Bound'] = df1.apply(lambda row: np.round((row.T1_4s+row.T1_6s)/row.T1_OversBat,2) , axis = 1)
df1['T2_Bound'] = df1.apply(lambda row: np.round((row.T2_4s+row.T2_6s)/row.T2_OversBat,2) , axis = 1)
# 5. avg wickets taken per over
df1['T1_Avg_Wkts_Taken'] = df1.apply(lambda row: np.round(row.T1_WktsTaken/row.T1_OversBowled,2) , axis = 1)
df1['T2_Avg_Wkts_Taken'] = df1.apply(lambda row: np.round(row.T2_WktsTaken/row.T2_OversBowled,2) , axis = 1)
# 6. avg wickets lost
df1['T1_Avg_Wkts_Lost'] = df1.apply(lambda row: np.round(row.T1_WktsLost/row.T1_OversBat,2) , axis = 1)
df1['T2_Avg_Wkts_Lost'] = df1.apply(lambda row: np.round(row.T2_WktsLost/row.T2_OversBat,2) , axis = 1)
# remove unvanted columns
data = df1.drop([ 
       'T1_BowlsFace', 'T1_OversBowl',
       'T1_RunsScored', 'T1_4s', 'T1_6s', 'T1_RunsConceded', 'T1_WktsTaken',
       'T1_WktsLost',   'T2_BowlsFace', 'T2_OversBowl',
       'T2_RunsScored', 'T2_4s', 'T2_6s', 'T2_RunsConceded', 'T2_WktsTaken',
       'T2_WktsLost',   'T1_OversBat', 'T2_OversBat',
       'T1_OversBowled', 'T2_OversBowled',  
], axis=1)
# save the dataset
data.to_csv('historical-featured.csv')